### sjoin_count
__About__
 - Function calculates the number of times each entity from a particular layer is overlapped with another layer entities.

__Input__
- target_layer, other_layer --> two GeoDataFrames objects, the first GeoDataFrame will be the target 
dataset.
- id1 --> (String obj type) name of id feature in target dataset.

__Output__
- A GeoDataFrame object, which is the target dataset, with one added 'count' feature,
which contains the number of times each entity from the target dataset intersects with the entities from the other dataset.

In [1]:
import geopandas as gpd
import pandas as pd

In [50]:
def sjoin_count(target_layer, other_layer,id1):
    #Spatial join
    joined = gpd.sjoin(target_layer, other_layer)
    joined['counter'] = None
    joined['counter'] = joined.groupby(id1)[id1].transform('count')
    joined= joined.drop_duplicates(subset= id1, keep='first')
    
    target_layer['count'] = 0
    
    for idx, row in joined.iterrows():
        indexes= target_layer.index[target_layer['geometry'] == row['geometry']].tolist()
        target_layer.at[indexes, 'count']=row['counter']
        
    return target_layer

In [56]:
# upload data into GeoDataFrames objects
business= gpd.read_file(r'C:\Users\Shaked Chen\Desktop\layers\1.shp',encoding= 'utf-8')
people_location= gpd.read_file(r'C:\Users\Shaked Chen\Desktop\layers\points.shp',encoding= 'utf-8')
people_location['id']= people_location.index
#business['OBJECTID'] = business.index

In [57]:
business = business.to_crs(people_location.crs)
#business= business.drop(['descriptio','sub_class','CLASS','BLDG_NUM','BLDG_LETT','NUM_ENTER','entity','symbol','Shape_Area','Shape_Le_1','fin_source','BLDG_LETT2','STREET_N_3','BLDG_NUM2','BLDG_LETT3','NUM_FLOORS','NUM_APTS','STREET_N_2','street_cod','Shape_Leng','campus','NUM_BUSNS','BLDG_NUM3'],axis=1)

In [59]:
s=sjoin_count(business, people_location,'OBJECTID')
s.to_excel(r'C:\Users\Shaked Chen\Desktop\ShakedGit\IDF\output.xls')

In [60]:
s['count'].value_counts()

0    245
1      7
3      2
9      1
7      1
4      1
2      1
Name: count, dtype: int64

RangeIndex(start=0, stop=258, step=1)